In [26]:
%pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.


In [27]:
from elasticsearch import Elasticsearch
import fitz
import os

In [28]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=["elastic","uxVFMV452qfPJkXeYlVW"],
    ca_certs="C:\\Users\\aonno\\Downloads\\elasticsearch-8.13.4-windows-x86_64\\elasticsearch-8.13.4\\config\\certs\\http_ca.crt"
)

es.ping()

True

In [29]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return "Error getting the file"

In [30]:
# Function to check if a file has already been ingested
def is_file_ingested(file_name, es_client, es_index):
    query = {
        "query": {
            "term": {
                "file_name.keyword": file_name
            }
        }
    }
    res = es_client.search(index=es_index, body=query)
    return len(res['hits']['hits']) > 0

In [31]:
es_index = "pdf_documents"

In [32]:
# Function to prepare data for Elasticsearch
def prepare_data_for_elasticsearch(pdf_folder):
    data_list = []
    for root, _, files in os.walk(pdf_folder):
        for file in files:
            if file.lower().endswith('.pdf'):
                if not is_file_ingested(file, es, es_index):
                    pdf_path = os.path.join(root, file)
                    text_content = extract_text_from_pdf(pdf_path)
                    data = {
                        "file_name": file,
                        "location": pdf_path,
                        "content": text_content
                    }
                    data_list.append(data)
                else:
                    print("file already ingested.Skipping")
    return data_list


In [33]:
# Function to index data into Elasticsearch
def index_data_to_elasticsearch(data_list, es_index, es_client):
    for data in data_list:
        es_client.index(index=es_index, body=data)

In [34]:
es.ping()

True

In [35]:
pdf_folder = ".\pdf"  # Replace with the path to your PDF folder
es_host = "http://localhost:9200"  # Replace with your Elasticsearch host
es_index = "pdf_documents"  # Replace with your Elasticsearch index name

# Initialize Elasticsearch client
# es_client = Elasticsearch([es_host])
# es is es client

# Extract data from PDFs and prepare for Elasticsearch
data_list = prepare_data_for_elasticsearch(pdf_folder)

# Index data into Elasticsearch
index_data_to_elasticsearch(data_list, es_index, es)

print(f"Indexed {len(data_list)} documents into Elasticsearch index '{es_index}'.")


file already ingested.Skipping
file already ingested.Skipping
Indexed 0 documents into Elasticsearch index 'pdf_documents'.


<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\aonno\AppData\Local\Temp\ipykernel_29940\3510188874.py:1: SyntaxWarning: invalid escape sequence '\p'
  pdf_folder = ".\pdf"  # Replace with the path to your PDF folder


In [36]:
# ### creta new index in elastic search
# from indexMapping import indexMapping
# try:
#     es.indices.create(index="all_products", mappings=indexMapping)
# except Exception as e:
#     print(e)

In [37]:
input_search_term = "Maritime"
# print("searching...")
# print(vector_of_input_keyword)

body = {
        "query": {
            "multi_match": {
                "query": input_search_term,
                "fields": ["content", "file_name"]
            }
        },
       }
res = es.search(index=es_index, body=body)

results = res["hits"]["hits"]
print(results)


[{'_index': 'pdf_documents', '_id': 'rOh5t48Bs1_zjc2DYD1k', '_score': 0.398678, '_ignored': ['content.keyword'], '_source': {'file_name': 'inec.pdf', 'location': '.\\pdf\\inec.pdf', 'content': "Author’s Biography \nAybars Oruc is a Marine Engineer. After he had worked as an engineer on different types of merchant vessels such as \naframax tanker, LPG tanker and container vessel, he worked as HSEQ Coordinator, and then HSEQ Superintendent in a \ntanker management company for 4 years. He is a Ph.D. candidate at the NTNU. His research field is maritime cyber security. \nClaims of State-Sponsored Cyberattack in the Maritime Industry \nA Oruc* M.Sc. MIET MIMarEST \n* Norwegian University of Science and Technology, Norway\n* Corresponding Author. Email: aybars.oruc@ntnu.no\nSynopsis \nDevelopments in technology bring inherent risks along with convenience. Undoubtedly, cyberattacks \nconstitute one potentially serious risk. While a stereotypical scenario involves a curious teenager sitting in